<a href="https://colab.research.google.com/github/Vish4github/Projects/blob/master/Sentiment_Classification_NB_%7C_Logistic_%7C_NNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%pip install fastai --upgrade

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import*
from fastai.text import*
from fastai.vision import *

In [0]:
import sklearn.feature_extraction.text as sklearn_text

In [0]:
#?? URLs

In [7]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [0]:
df=pd.read_csv(path/'texts.csv')

In [9]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [10]:
movie_reviews = (TextList.from_csv(path,'texts.csv',cols='text').split_from_df(col=2).label_from_df(cols=0))  #can help split data into test , validation etc - Here we are getting rid of col2 and saying col 0 is the label column

In [11]:
movie_reviews.valid.x[0],movie_reviews.valid.y[0]

(Text [ 2  5 21 71 ... 15  5  0 52], Category 1)

In [12]:
len(movie_reviews.train.x),len(movie_reviews.valid.x)

(800, 200)

In [13]:
len(movie_reviews.vocab.itos),len(movie_reviews.vocab.stoi)  #itos - integer to string and stoi - string to integer; the reason the nos are different - many words would be mapped as unknown, many as capital etc

(6008, 19159)

In [14]:
movie_reviews.vocab.itos[:10]  #based on frequency - what is itos - integer to strings - the movie reviews are stored as numbers(since the algo uses numbers and not strings) so if we want to see the actual words we need to use itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the']

In [15]:
  c= Counter([4,2,8,8,8,4])  #creates a dictionary with values
  c.values()


dict_values([2, 1, 3])

In [16]:
c.keys()

dict_keys([4, 2, 8])

In [17]:
c

Counter({2: 1, 4: 2, 8: 3})

A matrix with lots of zeroes is called . For sparse matrices a lot of memory can be saved

In [0]:
def get_doc_term_matrix(text_list, n_terms):
    
    # inputs:
    #    text_list, a TextList object
    #    n_terms, the number of tokens in our IMDb vocabulary
    
    # output: 
    #    the CSR format sparse representation of the document-term matrix in the form of a
    #    scipy.sparse.csr.csr_matrix object

    
    # initialize arrays
    values = []
    column_indices = []
    row_pointer = []
    row_pointer.append(0)

    # from the TextList object
    for _, doc in enumerate(text_list):
        feature_counter = Counter(doc.data)
        column_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        # Tack on N (number of nonzero elements in the matrix) to the end of the row_pointer array
        row_pointer.append(len(values))
        
    return scipy.sparse.csr_matrix((values, column_indices, row_pointer),
                                   shape=(len(row_pointer) - 1, n_terms),
                                   dtype=int)

In [19]:
%%time
train_doc_term = get_doc_term_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))
val_doc_term = get_doc_term_matrix(movie_reviews.valid.x,len(movie_reviews.vocab.itos))

CPU times: user 179 ms, sys: 4.77 ms, total: 184 ms
Wall time: 184 ms


NAIVE BAYES

In [20]:
movie_reviews.y.classes

['negative', 'positive']

In [0]:
x = train_doc_term
y = movie_reviews.train.y
val_y = movie_reviews.valid.y
v = movie_reviews.vocab

In [0]:
positive = y.c2i['positive']
negative =y.c2i['negative']

In [23]:
x.shape

(800, 6008)

In [0]:
import numpy as np
#to calculate the log count ratios

p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))   
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [25]:
p0[v.stoi['loved']],p1[v.stoi['loved']]  # no of times word loved appeared in negative and positive reviews

(12, 29)

In [26]:
p0[v.stoi['hate']],p1[v.stoi['hate']] 

(13, 5)

In [27]:
v.stoi['hated']  # the integer value of word hate 

1977

In [28]:
a = np.argwhere((x[:,1977]>0))[:,0];a    #rows or reviews from the dtm where the count of word hate > 1

array([ 15,  49, 304, 351, 393, 612, 695, 773], dtype=int32)

In [29]:
b = np.argwhere(y.items==positive)[:,0];b    #rows corresponding to positive reviews

array([  1,   3,  10,  11, ..., 787, 789, 790, 797])

In [30]:
set(a).intersection(set(b))  # rows where the review is positive but it has the word hate

{393, 612, 695}

In [31]:
review = movie_reviews.train.x[393];
print(''.join(review.text))

xxbos xxmaj there are numerous films relating to xxup xxunk , but xxmaj mother xxmaj night is quite xxunk among them : xxmaj in this film , we are introduced to xxmaj howard xxmaj campbell ( xxmaj nolte ) , an xxmaj american living in xxmaj berlin and married to a xxmaj german , xxmaj xxunk xxmaj xxunk ( xxmaj lee ) , who decides to accept the role of a spy : xxmaj more specifically , a xxup cia agent xxmaj major xxmaj xxunk ( xxmaj goodman ) recruits xxmaj campbell who becomes a xxmaj nazi xxunk in order to enter the highest xxunk of the xxmaj hitler xxunk . xxmaj however , the deal is that the xxup us xxmaj government will never xxunk xxmaj campbell 's role in the war for national security reasons , and so xxmaj campbell becomes a hated figure across the xxup us . xxmaj after the war , he tries to xxunk his identity , but the past comes back and xxunk him . xxmaj his only " friend " is xxmaj xxunk , but even he can not do much for the xxunk of events that fall upon poor xxmaj campbel

Applying Naive Bayes

In [32]:
pr1 = (p1+1)/((y.items==positive).sum()+1)  #adding 1 in num and deno for numerical stability
pr0 = (p0+1)/((y.items==negative).sum()+1)

r = np.log(pr1/pr0);r

array([-0.015348,  0.084839,  0.      ,  0.084839, ...,  1.471133, -1.301455, -1.301455, -1.301455])

Vocab most likely associated with postive and negative reviews

In [0]:
biggest = np.argpartition(r,-10)[-10:]
smallest = np.argpartition(r,10)[:10]

In [34]:
#most positive
[v.itos[k] for k in biggest]

['han',
 'jabba',
 'gilliam',
 'davies',
 'noir',
 'felix',
 'jimmy',
 'astaire',
 'fanfan',
 'biko']

In [35]:
np.argmax(train_doc_term[:,v.stoi['jimmy']])

741

In [36]:
movie_reviews.train.x[741].text  #jimmy appears multiple times in this positive review

'xxbos * * * xxup spoilers * * * xxmaj all too , in real life as well as in the movies , familiar story that happens to many young men who are put in a war zone with a gun , or xxunk , in their hands . xxmaj the case of young and innocent , in never xxunk or firing a gun , xxmaj jimmy xxmaj davis , xxmaj xxunk xxmaj tone , has been repeated thousands of times over the xxunk when men , like xxmaj jimmy xxmaj davis , are forced to take up arms for their country . \n \n  xxmaj jimmy who at first wanted to be kicked out of the xxup us xxmaj army but was encouraged to stay , by being xxunk in the mouth , by his good friend xxmaj fred xxup xxunk xxmaj willis , xxmaj xxunk xxmaj tracy , ended up on the front lines in xxmaj france . xxmaj with xxmaj jimmy \'s unit xxunk down by a xxmaj german machine gun nest he single xxunk put it out of xxunk picking off some half dozen xxmaj german soldiers from the xxunk of a nearby church xxunk . xxmaj it was when xxmaj jimmy xxunk down the last surviving

In [0]:
b = np.log((y.items==positive).mean()/(y.items==negative).mean())  #ratio of avg postive reviews to neg reviews

In [0]:
preds = (val_doc_term @ r + b) > 0 

In [39]:
(preds == val_y.items).mean()

0.645

Binarized Naive Bayes - maybe it only matters to know whether a word is in the review or not and not the frequency 

In [0]:
x = train_doc_term.sign()
y= movie_reviews.train.y

In [0]:
negative = y.c2i['negative']
positive = y.c2i['positive']

In [0]:
p1 = np.squeeze(np.asarray(x[y.items==positive].sum(0)))
p0 = np.squeeze(np.asarray(x[y.items==negative].sum(0)))

In [0]:
pr1 = (p1+1)/((y.items==positive).sum()+1)
pr0 = (p0+1)/((y.items==negative).sum()+1)

In [0]:
r = np.log(pr1/pr0)
b = np.log((y.items==positive).mean()/(y.items==negative).mean())
preds = (val_doc_term @ r +b) >0

In [45]:
(preds == val_y.items).mean()

0.66

Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression

In [47]:
m= LogisticRegression(C=0.1,dual=False,solver='liblinear')
m.fit(x,y.items.astype(int))
preds=m.predict(val_doc_term)
(preds==val_y.items).mean()

0.685

Binarized version

In [48]:
m= LogisticRegression(C=0.1,dual=False,solver='liblinear')
m.fit(train_doc_term.sign(),y.items.astype(int))
preds=m.predict(val_doc_term.sign())
(preds==val_y.items).mean()

0.83

With n-grams

In [49]:
vocab_sample = movie_reviews.vocab.itos
vocab_len = len(vocab_sample)
print(f'IMDb_sample vocabulary has {vocab_len} tokens')

IMDb_sample vocabulary has 6008 tokens


n gram token matrix

In [0]:
min_n=1
max_n=3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

In [51]:
%%time
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) not in ngramtoi:
                if len(ngram)==1:
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

CPU times: user 2min 50s, sys: 424 ms, total: 2min 51s
Wall time: 2min 51s


In [52]:
%%time
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

CPU times: user 205 ms, sys: 0 ns, total: 205 ms
Wall time: 205 ms


n gram doc matrix for validation data

In [53]:
%%time
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

CPU times: user 39.6 s, sys: 33 ms, total: 39.7 s
Wall time: 39.7 s


In [54]:
%%time
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

CPU times: user 39.2 ms, sys: 2 µs, total: 39.2 ms
Wall time: 39.7 ms


In [0]:
# will save these files as they are slow to be constructed every time

scipy.sparse.save_npz("train_ngram_matrix.npz", train_ngram_doc_matrix)
scipy.sparse.save_npz("valid_ngram_matrix.npz", valid_ngram_doc_matrix)

with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('ngramtoi.pickle', 'wb') as handle:
    pickle.dump(ngramtoi, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# lets load these n grams now

train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_matrix.npz")


with open('itongram.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
with open('ngramtoi.pickle', 'rb') as handle:
    b = pickle.load(handle)

Naive Bayes using N grams

In [0]:
x = train_ngram_doc_matrix  #dtm 
y = movie_reviews.train.y  # train subset of responses

In [58]:
positive = y.c2i['positive']
negative = y.c2i['negative']
negative

0

In [0]:
x
k=260382

In [60]:
pos = (y.items==positive)[:k]  #positive subset - which all reviews are positive or negatively tagged in the train datset
neg = (y.items == negative)[:k]  #negative subset
neg

array([ True, False,  True, False, ...,  True, False,  True,  True])

In [0]:
xx = x[:k]

In [0]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]
#valid_labels

In [0]:
p0 = np.squeeze(np.asarray(xx[neg].sum(0)))   #based on the dtm - will get the reviews which are positively or negativey tagged based on the index position from the responses
p1 = np.squeeze(np.asarray(xx[pos].sum(0)))
pr1 = (p1+1)/((y.items==positive).sum()+1)
pr0 = (p0+1)/((y.items==negative).sum()+1)
r = np.log(pr1/pr0)

In [0]:
b = np.log((y.items==positive).mean()/(y.items==negative).mean())   #based on the responses (actual data)

In [0]:
pre_preds = (valid_ngram_doc_matrix @ r.T + b) 

In [66]:
preds = pre_preds.T>0
preds

array([ True,  True,  True,  True, ...,  True, False, False,  True])

In [67]:
(preds==valid_labels).mean()

0.76

Logistic Regression with n grams

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer


In [0]:
veczr = CountVectorizer(ngram_range=(1,3), preprocessor=noop, tokenizer=noop, max_features=800000)

In [0]:
train_docs = movie_reviews.train.x
train_words = [[movie_reviews.vocab.itos[o] for o in doc.data] for doc in train_docs]

In [0]:
valid_docs = movie_reviews.valid.x
valid_words = [[movie_reviews.vocab.itos[o] for o in doc.data] for doc in valid_docs]

In [72]:
%%time
train_ngram_doc_matrix_veczr = veczr.fit_transform(train_words)
train_ngram_doc_matrix_veczr

CPU times: user 1.53 s, sys: 9 ms, total: 1.54 s
Wall time: 1.54 s


In [73]:
valid_ngram_doc_matrix_veczr = veczr.transform(valid_words)
valid_ngram_doc_matrix_veczr

<200x260381 sparse matrix of type '<class 'numpy.int64'>'
	with 93552 stored elements in Compressed Sparse Row format>

In [0]:
vocab = veczr.get_feature_names()

In [75]:
m = LogisticRegression(C=0.1, dual=False, solver = 'liblinear')
m.fit(train_ngram_doc_matrix_veczr, y.items);

preds = m.predict(valid_ngram_doc_matrix_veczr)
accuracy =(preds==valid_labels).mean()
print(f'Accuracy  = {accuracy} for Logistic Regression, with full trigram counts from `CountVectorizer`' )

Accuracy  = 0.78 for Logistic Regression, with full trigram counts from `CountVectorizer`


In [76]:
# Using our dtm

m2 = LogisticRegression(C=0.1, dual=False,solver='liblinear')
m2.fit(train_ngram_doc_matrix, y.items)
preds = m2.predict(valid_ngram_doc_matrix)
accuracy = (preds==valid_labels).mean()
print(f'Accuracy  = {accuracy} for Not-Binarized Logistic Regression, with our Trigrams' )

Accuracy  = 0.795 for Not-Binarized Logistic Regression, with our Trigrams
